In [ ]:
import numpy as np
from scipy.constants import g

from LNECSPA import LTFdb

def ltf_to_txt(file_path):
    tgtA = LTFdb()
    tgtA.read(file_path)

    # 1. Identify channel indices
    disp_inds = [i for i, unit in enumerate(tgtA.units) if unit.lower() in ('mm', 'm')]
    acc_inds  = [i for i, typ  in enumerate(tgtA.types) if typ.lower() == 'acceleration']

    # 2. Build time vector
    dt     = float(tgtA.dt[0])
    n_pts  = tgtA.data.shape[1]
    time_s = np.arange(n_pts) * dt

    # 3. Convert to SI
    disp_si = tgtA.data[disp_inds].astype(float)
    for idx, ch in enumerate(disp_inds):
        if tgtA.units[ch].lower() == 'mm':
            disp_si[idx] *= 1e-3

    acc_si = tgtA.data[acc_inds].astype(float)
    for idx, ch in enumerate(acc_inds):
        if tgtA.units[ch].lower() == 'g':
            acc_si[idx] *= g

    # 4. Stack columns: time | displacements | accelerations
    out_arr = np.vstack((time_s, disp_si, acc_si)).T

    # 5. Remove columns that are entirely zero
    mask = ~np.all(np.isclose(out_arr, 0.0), axis=0)
    out_arr = out_arr[:, mask]

    # 6. Prepare headers aligned with filtered columns
    header_cols = ['time_s'] + \
        [f"{tgtA.names[i]}_m" for i in disp_inds] + \
        [f"{tgtA.names[i]}_m_per_s2" for i in acc_inds]
    header_cols = [h for h, m in zip(header_cols, mask) if m]

    # 7. Export to fixed-width .txt file
    width = 15
    fmt = ['%15.6e'] * len(header_cols)
    header_line = ''.join(h.center(width) for h in header_cols)

    txt_filename = 'tgtA_SI_time_disp_acc.txt'
    with open(txt_filename, 'w') as f:
        f.write(header_line + '\n')
        np.savetxt(f, out_arr, fmt=fmt)

    print(f"Exported {out_arr.shape[0]} samples and {out_arr.shape[1]} columns to '{txt_filename}'.")


In [2]:
file_path = r'C:\Users\afons\OneDrive - Universidade de Lisboa\Controlo de Plataforma Sismica\LNEC_Adapta_Driver\2024_ERIES_RE-Safe\Targets\LAquilaReducedScale.tgt'
ltf_to_txt(file_path)

Exported 8982 samples and 3 columns to 'tgtA_SI_time_disp_acc.txt'.


In [3]:
file_path = r'C:\Users\afons\OneDrive - Universidade de Lisboa\Controlo de Plataforma Sismica\LNEC_Adapta_Driver\2024_ERIES_RE-Safe\TestLAquila\2024-09-09\LAquilaReducedScale_0.DRV'
ltf_to_txt(file_path)

Exported 8982 samples and 2 columns to 'tgtA_SI_time_disp_acc.txt'.
